<a href="https://colab.research.google.com/github/DevDope/Abracadabra/blob/main/Chroma_db.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip uninstall torch torchvision torchaudio -y
!pip install torch==2.4.1
!pip install llama_index
%pip install llama-index-vector-stores-chroma
!pip install chromadb
!pip install llama-index-multi-modal-llms-ollama
!pip install llama-index-readers-file
!pip install unstructured
!pip install llama-index-embeddings-huggingface
!pip install llama-index-vector-stores-qdrant
!pip install llama-index-embeddings-clip
!pip install llama-index qdrant_client torch transformers
!pip install llama-index-llms-ollama
!pip install tqdm



In [ ]:
from google.colab import drive
import json
import chromadb
from tqdm import tqdm
from llama_index.vector_stores.chroma import ChromaVectorStore

drive.mount('/content/drive')


file_path = '/content/drive/My Drive/final_milliondataset_with_embeddings.json'
chromadb_path = '/content/drive/My Drive/chroma_db'


batch_size = 10000


db = chromadb.PersistentClient(path=chromadb_path)
chroma_collection = db.get_or_create_collection("music_recommendation")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)


doc_counter = 0

with open(file_path, 'r') as file:
    batch_texts = []
    batch_embeddings = []
    batch_metadatas = []
    batch_ids = []

    for line in tqdm(file, desc="Loading documents"):

        json_object = json.loads(line)


        text = json_object['text']
        embedding = json_object['embedding']


        similar_songs = json_object.get('Similar Songs', [])
        similar_songs_str = ", ".join(
            [f'"{song.get("Similar Song 1", "")}" by {song.get("Similar Artist 1", "")}' for song in similar_songs]
        )


        metadata = {
            k: (v if v is not None else "") for k, v in json_object.items() if k not in ['text', 'embedding', 'Similar Songs']
        }
        metadata['Similar Songs'] = f"The similar songs are {similar_songs_str}."


        doc_id = f"doc_{doc_counter}"
        doc_counter += 1


        batch_texts.append(text)
        batch_embeddings.append(embedding)
        batch_metadatas.append(metadata)
        batch_ids.append(doc_id)


        if len(batch_texts) >= batch_size:

            chroma_collection.add(
                documents=batch_texts,
                embeddings=batch_embeddings,
                metadatas=batch_metadatas,
                ids=batch_ids
            )

            batch_texts = []
            batch_embeddings = []
            batch_metadatas = []
            batch_ids = []


    if batch_texts:
        chroma_collection.add(
            documents=batch_texts,
            embeddings=batch_embeddings,
            metadatas=batch_metadatas,
            ids=batch_ids
        )


db2 = chromadb.PersistentClient(path=chromadb_path)
chroma_collection = db2.get_or_create_collection("music_recommendation")
vector_store = ChromaVectorStore(chroma_collection=chroma_collection)



